In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [3]:
indices = ["^GSPC.csv", "^N225.csv", "^SSEC.csv", "^HSI.csv", "^BSESN.csv", "^SSMI.csv", "^BVSP.csv"]

data = []
for index in indices:
    df = pd.read_csv("data/"+index, index_col='Date')
    df_t = pd.concat([df['Close'],
                        df['Close']/df['Close'][-1],
                        df['Close'].pct_change(),
                        df['Close'].pct_change().rolling(10).std()], axis=1).dropna()
    df_t.columns = ["Price", "Norm", "Change", "Volume"]
    data.append(df_t)

data[5] = data[5].loc['1990-11-09':,:]